## ETCD LAB 

A distributed, reliable key-value store for the most critical data of a distributed system.  
Homepage: https://etcd.io/

Key features:

- Simple: well-defined, user-facing API (gRPC)
- Secure: automatic TLS with optional client cert authentication
- Fast: benchmarked 10,000 writes/sec
- Reliable: properly distributed using Raft


There are two major use cases: concurrency control in the distributed system and application configuration store. For example, CoreOS Container Linux uses etcd to achieve a global semaphore to avoid that all nodes in the cluster rebooting at the same time. Also, Kubernetes use etcd for their configuration store.

During this lab we will be using etcd3 python client.  
Homepage: https://pypi.org/project/etcd3/

Etcd credentials are shared on the slack channel: https://join.slack.com/t/ibm-agh-labs/shared_invite/zt-e8xfjgtd-8IDWmn912qPOflbM1yk6~Q

Please copy & paste them into the cell below:

In [66]:
etcdCreds = {}  # copy and paste etcd credentials provided on the Slack channel here

In [2]:
!pip install etcd3

### How to connect to etcd using certyficate (part 1: prepare file with certificate)

In [67]:
import base64
import tempfile

etcdHost = etcdCreds["connection"]["grpc"]["hosts"][0]["hostname"]
etcdPort = etcdCreds["connection"]["grpc"]["hosts"][0]["port"]
etcdUser = etcdCreds["connection"]["grpc"]["authentication"]["username"]
etcdPasswd = etcdCreds["connection"]["grpc"]["authentication"]["password"]
etcdCertBase64 = etcdCreds["connection"]["grpc"]["certificate"]["certificate_base64"]
                           
etcdCertDecoded = base64.b64decode(etcdCertBase64)
etcdCertPath = "{}/{}.cert".format(tempfile.gettempdir(), etcdUser)
                           
with open(etcdCertPath, 'wb') as f:
    f.write(etcdCertDecoded)

print(etcdCertPath)

/home/dsxuser/.tmp/ibm_cloud_f59f3a7b_7578_4cf8_ba20_6df3b352ab46.cert


### Short Lab description

During the lab we will simulate system that keeps track of logged users
- All users will be stored under parent key (path): /logged_users
- Each user will be represented by key value pair
    - key /logged_users/name_of_the_user
    - value hostname of the machine (e.g. name_of_the_user-hostname)

### How to connect to etcd using certyficate (part 2: create client)

In [68]:
import etcd3

etcd = etcd3.client(
    host=etcdHost,
    port=etcdPort,
    user=etcdUser,
    password=etcdPasswd,
    ca_cert=etcdCertPath
)

cfgRoot='/logged_users'

### Task 1 : Fetch username and hostname

define two variables
- username name of the logged user (tip: use getpass library)
- hostname hostname of your mcomputer (tip: use socket library)

In [21]:
import getpass
import socket

username = "kustra"  # You can put your name here, while this code is run in the container and user name would be same for all students
hostname = socket.gethostname()

userKey='{}/{}'.format(cfgRoot, username)
userKey, '->', hostname

etcd.put(userKey, hostname)

header {
  cluster_id: 17394822126184162018
  member_id: 17586574424884576738
  revision: 354954
  raft_term: 3204
}

In [22]:
value, metadata = etcd.get('/logged_users/kustra')
print(value)
print(metadata.key)
print(metadata.version)

b'notebook-0b34268899e14795b60b6d36d0883ae1-576b4b5d8d-w62xc'
b'/logged_users/kustra'
2


### Task 2 : Register number of users 

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

for all names in table fixedUsers register the appropriate key value pairs

In [13]:
fixed_users = [
    'Adam',
    'Borys',
    'Cezary',
    'Damian',
    'Emil',
    'Filip',
    'Gustaw',
    'Henryk',
    'Ignacy',
    'Jacek',
    'Kamil',
    'Leon',
    'Marek',
    'Norbert',
    'Oskar',
    'Patryk',
    'Rafał',
    'Stefan',
    'Tadeusz'
]

for user in fixed_users:
    user_key='{}/{}'.format(cfgRoot, user)
    hostname = socket.gethostname()
    print(user_key)
    etcd.put(user_key,hostname)

/logged_users/Adam
/logged_users/Borys
/logged_users/Cezary
/logged_users/Damian
/logged_users/Emil
/logged_users/Filip
/logged_users/Gustaw
/logged_users/Henryk
/logged_users/Ignacy
/logged_users/Jacek
/logged_users/Kamil
/logged_users/Leon
/logged_users/Marek
/logged_users/Norbert
/logged_users/Oskar
/logged_users/Patryk
/logged_users/Rafał
/logged_users/Stefan
/logged_users/Tadeusz


### Task 3: List all users

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

List all registered user (tip: use common prefix)

In [23]:
results = etcd.get_prefix(cfgRoot)
{m.key : v for v, m in results}

{b'/logged_users': b'/logged_users/Tadeusz - Registered',
 b'/logged_users//logged_users': b'notebook-438cad81f4974877a0c254a6d6ff0145-58f95b7948-zfpxd',
 b'/logged_users/0': b'Adam',
 b'/logged_users/1': b'Borys',
 b'/logged_users/10': b'Kamil',
 b'/logged_users/11': b'Leon',
 b'/logged_users/12': b'Marek',
 b'/logged_users/13': b'Norbert',
 b'/logged_users/14': b'Oskar',
 b'/logged_users/15': b'Patryk',
 b'/logged_users/16': b'Rafa\xc5\x82',
 b'/logged_users/17': b'Stefan',
 b'/logged_users/18': b'Tadeusz',
 b'/logged_users/2': b'Cezary',
 b'/logged_users/3': b'Damian',
 b'/logged_users/4': b'Emil',
 b'/logged_users/5': b'Filip',
 b'/logged_users/6': b'Gustaw',
 b'/logged_users/7': b'Henryk',
 b'/logged_users/8': b'Ignacy',
 b'/logged_users/9': b'Jacek',
 b'/logged_users/Adam': b'notebook-premium2py36f501191650b745849cf3df420b3c3644-79c52zpk9',
 b'/logged_users/Andrii': b'notebook-75760867122b4b86a5bbcd2f5ccc321b-66fd799d96-dc62c',
 b'/logged_users/Borys': b'notebook-premium2py36f501

### Task 4 : Same as Task2, but use transaction

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

for all names in table fixedUsers register the appropriate key value pairs, use transaction to make it a single request  
(Have you noticed any difference in execution time?)

In [24]:
etcd.transaction(
        compare=[etcd.transactions.version(cfgRoot) == 0],
        success=[etcd.transactions.put('{}/{}'.format(cfgRoot, user), 'user-{}'.format(user)) for user in fixed_users],
        failure=[etcd.transactions.put('/tmp/errors', 'condition failed')]
    )

(False, [response_put {
    header {
      revision: 354975
    }
  }])

### Task 5 : Get single key (e.g. status of transaction)

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

Check the key you are modifying in on-failure handler in previous task

In [26]:
for info in etcd.get_prefix('/tmp/errors'):
    print(info)

(b'condition failed', <etcd3.client.KVMetadata object at 0x7f65dc4d4710>)


### Task 6 : Get range of Keys (Emil -> Oskar) 

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

- Get range of keys
- Is it inclusive / exclusive?
- Sort the resposne descending
- Sort the resposne descending by value not by key

In [32]:
range_from = cfgRoot + '/' + 'Emil'
range_to = cfgRoot + '/' + 'Oskar'
for key in etcd.get_range(range_from, range_to, sort_order='descend', sort_target='value'):
    print(key)

(b'value first', <etcd3.client.KVMetadata object at 0x7f65dc4d4e10>)
(b'testmessage3', <etcd3.client.KVMetadata object at 0x7f65dc4d4da0>)
(b'test_9', <etcd3.client.KVMetadata object at 0x7f65dc4d4e10>)
(b'test', <etcd3.client.KVMetadata object at 0x7f65dc4d4da0>)
(b'notebook-condafree1py3690879dede28a40ed88e530f26cab9341-6cpwqdw', <etcd3.client.KVMetadata object at 0x7f65dc4d4780>)
(b'notebook-condafree1py3681d3d2d480474a18b8276572c8600ae7-68hmpxx', <etcd3.client.KVMetadata object at 0x7f65dc4d4da0>)
(b'notebook-a7823551d9b84fd3b085958036c5f597-d469f8875-ltf7p', <etcd3.client.KVMetadata object at 0x7f65dc4d4780>)
(b'notebook-66c43e4471114fa295d284b8afdefbb0-58bc66c8d6-9bxrd', <etcd3.client.KVMetadata object at 0x7f65dc4d4da0>)
(b'notebook-66c43e4471114fa295d284b8afdefbb0-58bc66c8d6-9bxrd', <etcd3.client.KVMetadata object at 0x7f65dc4d4780>)
(b'notebook-66c43e4471114fa295d284b8afdefbb0-58bc66c8d6-9bxrd', <etcd3.client.KVMetadata object at 0x7f65dc4d4da0>)
(b'notebook-66c43e4471114fa295

### Task 7: Atomic Replace

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

Do it a few times, check if value has been replaced depending on condition

In [42]:
for user in fixed_users:
    result = etcd.replace('{}/{}'.format(cfgRoot, user), user, 'new' + user)
    print(user, ':', result)

Adam : False
Borys : False
Cezary : False
Damian : False
Emil : False
Filip : False
Gustaw : False
Henryk : False
Ignacy : False
Jacek : False
Kamil : False
Leon : False
Marek : False
Norbert : False
Oskar : False
Patryk : False
Rafał : False
Stefan : False
Tadeusz : False


### Task 8 : Create lease - use it to create expiring key

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

You can create a key that will be for limited time
add user that will expire after a few seconds

Tip: Use lease


In [54]:
import time

key = cfgRoot + '/' + 'blaise10sec'
value = '10sec'
print(key)
lease = etcd.lease(ttl=2)

for user in etcd.get_prefix(key):
    print(user)
    
time.sleep(4)

print('after 4 seconds:')
for user in etcd.get_prefix(key):
    print(user)

/logged_users/blaise10sec
after 4 seconds:


### Task 9 : Create key that will expire after you close the connection to etcd

Tip: use threading library to refresh your lease

In [63]:
import threading

lease = etcd.lease(ttl=5)

key = cfgRoot + '/' + 'blaise10sec'
value = '10sec'
etcd.put(key, value, lease=lease)

def refreshLease():
    lease.refresh()
    threading.Timer(1, refreshLease).start()

refreshLease()

print(etcd.get(key))
time.sleep(10)
print(etcd.get(key))

(b'10sec', <etcd3.client.KVMetadata object at 0x7f65dc4da6d8>)
(b'10sec', <etcd3.client.KVMetadata object at 0x7f65dc4c4278>)


### Task 10: Use lock to protect section of code

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

In [75]:
with etcd.lock('my_lock', ttl=5) as lock:
    print('Is acquaired?', lock.is_acquired())
    # some code
    lock.acquire()
    lock.release()
    print('Is acquaired?', lock.is_acquired())
       

Is acquaired? True
Is acquaired? False


### Task 11: Watch key

etcd3 api: https://python-etcd3.readthedocs.io/en/latest/usage.html

This cell will lock this notebook on waiting  
After running it create a new notebook and try to add new user

In [74]:
def callb(cb):
    print(cb)

etcd.add_watch_callback(key='/users/test', callback=callb)

5

In [76]:
etcd.put('/users/test', 'blazej')

header {
  cluster_id: 17394822126184162018
  member_id: 2713756945860001751
  revision: 358045
  raft_term: 3204
}